In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix, classification_report

In [2]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

In [3]:
df[['zipcode', 'date']]

,zipcode,date
0,30002,2012-03-31
1,30002,2012-04-30
2,30002,2012-05-31
3,30002,2012-06-30
4,30002,2012-07-31
...,...,...
884087,34698,2023-08-31
884088,34698,2023-09-30
884089,34698,2023-10-31
884090,34698,2023-11-30


In [4]:
df_sorted = df.sort_values(['zipcode', 'date']).reset_index(drop=True)
df_sorted[['zipcode', 'date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [5]:
type(df_sorted['date'][0]), df_sorted['date'][0]

(str, '2012-03-31')

In [6]:
df_sorted['date'] = pd.to_datetime(df_sorted['date'])
df_sorted['date']

0        2012-03-31
1        2012-04-30
2        2012-05-31
3        2012-06-30
4        2012-07-31
            ...    
884087   2023-08-31
884088   2023-09-30
884089   2023-10-31
884090   2023-11-30
884091   2023-12-31
Name: date, Length: 884092, dtype: datetime64[ns]

In [7]:
df_grouped = df_sorted.groupby('zipcode')

In [8]:
ts_all = pd.DataFrame()

#ls = [1,3,6,12]

i = 1

#for i in ls:
ts_all[f'target_profitable_{i}m'] = (df_grouped['price'].shift(-i) > 1.02 * df_sorted['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_all['zipcode_obj'] = df_grouped['zipcode'].shift(0).astype('object') # but remains still an integer...
ts_all['date'] = df_sorted['date']
numeric_column_names = df_sorted.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.


for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)

ts_all = ts_all.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])

/tmp/ipykernel_43204/1715166150.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_43204/1715166150.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_all[f'{feature}_{lag}m'] = df_grouped[f'{feature}'].shift(lag)
/tmp/ipykernel_43204/1715166150.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.

In [9]:
#ts_all
numeric_column_names

Index(['median_sale_price', 'median_list_price', 'median_ppsf',
       'median_list_ppsf', 'homes_sold', 'pending_sales', 'new_listings',
       'inventory', 'median_dom', 'avg_sale_to_list', 'sold_above_list',
       'off_market_in_two_weeks', 'zipcode', 'year', 'bank', 'bus', 'hospital',
       'mall', 'park', 'restaurant', 'school', 'station', 'supermarket',
       'Total Population', 'Median Age', 'Per Capita Income',
       'Total Families Below Poverty', 'Total Housing Units', 'Median Rent',
       'Median Home Value', 'Total Labor Force', 'Unemployed Population',
       'Total School Age Population', 'Total School Enrollment',
       'Median Commute Time', 'price'],
      dtype='object')

In [10]:
df_sorted

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,89000.0,252250.0,65.441176,146.728867,3.0,6.0,6.0,12.0,179.0,...,1138.0,769.0,269800.0,1764.0,112.0,2933.0,2933.0,1493.0,195148.086474,Boston-Cambridge-Newton
1,2012-04-30,124000.0,229500.0,120.984975,124.798061,4.0,7.0,11.0,15.0,262.5,...,1138.0,769.0,269800.0,1764.0,112.0,2933.0,2933.0,1493.0,194446.137500,Boston-Cambridge-Newton
2,2012-05-31,177500.0,207450.0,120.984975,117.691290,6.0,3.0,14.0,17.0,269.5,...,1138.0,769.0,269800.0,1764.0,112.0,2933.0,2933.0,1493.0,194580.490265,Boston-Cambridge-Newton
3,2012-06-30,217000.0,203750.0,121.340580,127.536875,5.0,6.0,18.0,22.0,249.0,...,1138.0,769.0,269800.0,1764.0,112.0,2933.0,2933.0,1493.0,195081.437213,Boston-Cambridge-Newton
4,2012-07-31,130000.0,193900.0,81.948640,118.686333,5.0,9.0,18.0,22.0,222.0,...,1138.0,769.0,269800.0,1764.0,112.0,2933.0,2933.0,1493.0,196403.714599,Boston-Cambridge-Newton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884087,2023-08-31,502900.0,550000.0,305.790501,296.530498,137.0,126.0,133.0,62.0,12.0,...,8318.0,1677.0,497100.0,10143.0,395.0,19904.0,19904.0,7787.0,564276.110320,Portland-Vancouver-Hillsboro
884088,2023-09-30,509900.0,511755.0,311.643836,300.137268,139.0,112.0,110.0,64.0,16.5,...,8318.0,1677.0,497100.0,10143.0,395.0,19904.0,19904.0,7787.0,566663.763169,Portland-Vancouver-Hillsboro
884089,2023-10-31,525000.0,504450.0,306.527550,301.147755,135.0,95.0,94.0,61.0,19.0,...,8318.0,1677.0,497100.0,10143.0,395.0,19904.0,19904.0,7787.0,568266.780424,Portland-Vancouver-Hillsboro
884090,2023-11-30,525000.0,532580.0,309.415584,296.314311,118.0,78.0,84.0,70.0,30.5,...,8318.0,1677.0,497100.0,10143.0,395.0,19904.0,19904.0,7787.0,569327.047815,Portland-Vancouver-Hillsboro


In [9]:
#type(ts_all)
ts_all = ts_all.dropna()
ts_all

,target_profitable_1m,zipcode_obj,date,median_sale_price_1m,median_sale_price_2m,median_sale_price_3m,median_list_price_1m,median_list_price_2m,median_list_price_3m,median_ppsf_1m,...,Total School Age Population_3m,Total School Enrollment_1m,Total School Enrollment_2m,Total School Enrollment_3m,Median Commute Time_1m,Median Commute Time_2m,Median Commute Time_3m,price_1m,price_2m,price_3m
3,0,1431,2012-06-30,177500.0,124000.0,89000.0,207450.0,229500.0,252250.0,120.984975,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,194580.490265,194446.137500,195148.086474
4,0,1431,2012-07-31,217000.0,177500.0,124000.0,203750.0,207450.0,229500.0,121.340580,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,195081.437213,194580.490265,194446.137500
5,0,1431,2012-08-31,130000.0,217000.0,177500.0,193900.0,203750.0,207450.0,81.948640,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,196403.714599,195081.437213,194580.490265
6,0,1431,2012-09-30,130000.0,130000.0,217000.0,203700.0,193900.0,203750.0,95.205795,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,197743.074185,196403.714599,195081.437213
7,0,1431,2012-10-31,191000.0,130000.0,130000.0,162537.5,203700.0,193900.0,118.367237,...,2933.0,2933.0,2933.0,2933.0,1493.0,1493.0,1493.0,198843.413326,197743.074185,196403.714599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884087,0,98686,2023-08-31,515000.0,530900.0,523227.5,527450.0,520000.0,474900.0,305.216752,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,561259.870706,558047.224949,555511.734585
884088,0,98686,2023-09-30,502900.0,515000.0,530900.0,550000.0,527450.0,520000.0,305.790501,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,564276.110320,561259.870706,558047.224949
884089,0,98686,2023-10-31,509900.0,502900.0,515000.0,511755.0,550000.0,527450.0,311.643836,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,566663.763169,564276.110320,561259.870706
884090,0,98686,2023-11-30,525000.0,509900.0,502900.0,504450.0,511755.0,550000.0,306.527550,...,19904.0,19904.0,19904.0,19904.0,7787.0,7787.0,7787.0,568266.780424,566663.763169,564276.110320


In [10]:
type(df_sorted['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
df_sorted[['zipcode','date']]

,zipcode,date
0,1431,2012-03-31
1,1431,2012-04-30
2,1431,2012-05-31
3,1431,2012-06-30
4,1431,2012-07-31
...,...,...
884087,98686,2023-08-31
884088,98686,2023-09-30
884089,98686,2023-10-31
884090,98686,2023-11-30


In [12]:
def split_train_predict(df):
    # Ensure it's sorted by zipcode and date
    df = df.sort_values(['zipcode_obj', 'date'])

    # Group by zipcode
    train_list = []
    predict_list = []

    for zipcode, group in df.groupby('zipcode_obj'):
        train_part = group.iloc[:-1]
        predict_part = group.iloc[-1:]

        train_list.append(train_part)
        predict_list.append(predict_part)

    # Concatenate the results
    df_train = pd.concat(train_list).reset_index(drop=True)
    df_predict = pd.concat(predict_list).reset_index(drop=True)

    return df_train, df_predict

# Apply the split
df_train, df_predict = split_train_predict(ts_all)

# Optional: verify
print("Train set:", df_train.shape, 'Type', type(df_train))
print("Predict set:", df_predict.shape, 'Type', type(df_predict))

Train set: (859188, 108) Type <class 'pandas.core.frame.DataFrame'>
Predict set: (6226, 108) Type <class 'pandas.core.frame.DataFrame'>


In [13]:
zipcode_identifier = df_predict['zipcode_obj']
zipcode_identifier

0        1431
1        1432
2        1450
3        1460
4        1463
        ...  
6221    98682
6222    98683
6223    98684
6224    98685
6225    98686
Name: zipcode_obj, Length: 6226, dtype: object

In [14]:
# Load your data, step 1
df = df_train

# Step 2: Define target and features
target = f'target_profitable_{i}m'
y = df[target]
X = df.drop(columns=[target, 'date'])

# Step 3: Check class distribution
print("Class distribution in full dataset:")
print(y.value_counts())


Class distribution in full dataset:
target_profitable_1m
0    820434
1     38754
Name: count, dtype: int64


In [15]:
#X

In [16]:
df_predict=df_predict.drop(columns=[f'target_profitable_{i}m', 'date'])


In [17]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6226 entries, 0 to 6225
Columns: 106 entries, zipcode_obj to price_3m
dtypes: float64(105), object(1)
memory usage: 5.0+ MB


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import xgboost as xgb

import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor


In [19]:
scaler = MinMaxScaler()

In [20]:
scaler.fit(X)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [21]:
#X_train_scaled = scaler.fit_transform(X)
X_train_scaled = scaler.transform(X)

In [22]:
X_predict_scaled = scaler.transform(df_predict)

In [23]:

# Step 7: Train XGBoost classifier
xgb_model = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)


In [24]:
xgb_model.fit(X_train_scaled, y)

# Step 8: Predict and evaluate
y_pred = xgb_model.predict(X_predict_scaled)

'''# Step 9: Evaluation
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))'''

/home/jensk/.pyenv/versions/3.10.6/envs/zillows_real_estate/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [17:26:39] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


'# Step 9: Evaluation\nprint("\nConfusion Matrix:")\nprint(confusion_matrix(y_test, y_pred))\n\nprint("\nClassification Report:")\nprint(classification_report(y_test, y_pred))'

In [25]:
import os
from datetime import datetime
import joblib

In [26]:
# Define model save path
#model_dir = "/home/jessi/code/JensKlug/zillows_real_estate/zillow/model/models"
model_dir = "/home/jensk/code/JensKlug/zillows_real_estate/zillow/model/models"
os.makedirs(model_dir, exist_ok=True)

# Timestamp for versioning
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
model_path = os.path.join(model_dir, f"{timestamp}_xgboost_model{i}.joblib")

# Save pipeline
joblib.dump(xgb_model, model_path)

print(f"\n✅ Model saved to: {model_path}")


✅ Model saved to: /home/jensk/code/JensKlug/zillows_real_estate/zillow/model/models/20250613-172652_xgboost_model1.joblib


In [27]:
#y_pred = xgb_model.predict(X_predict_scaled)

In [28]:
np.savetxt(f"output_{i}m.csv", y_pred, delimiter=",", header=f"target_-{i}m", comments='')

#del y_pred

In [29]:
np.savetxt("zipcode_identifier.csv", zipcode_identifier, delimiter=",", header="zipcode", comments='')

In [30]:
pd_y_pred = pd.DataFrame(y_pred)

In [31]:
pd_y_pred.value_counts()

0
0    6221
1       5
Name: count, dtype: int64

In [32]:
'''
ts_train = pd.DataFrame()

ls = [1,3,6,12]

i = 1

#for i in ls:
ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * df_train['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_train['zipcode_obj'] = grouped_train['zipcode'].shift(0).astype('object') # but remains still an integer...
numeric_column_names = df_train.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.


for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

ts_train = ts_train.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])
'''

"\nts_train = pd.DataFrame()\n\nls = [1,3,6,12]\n\ni = 1\n\n#for i in ls:\nts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * df_train['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!\nts_train['zipcode_obj'] = grouped_train['zipcode'].shift(0).astype('object') # but remains still an integer...\nnumeric_column_names = df_train.select_dtypes(include='number').columns # drop the lags of zipcode-> this in Jessica's code.\n\n\nfor feature in numeric_column_names:\n    for lag in range(1, 4):  # 1 to 3 months\n        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)\n\nts_train = ts_train.drop(columns=['zipcode_1m', 'zipcode_2m', 'zipcode_3m'])\n"

In [33]:
'''ts_test = pd.DataFrame()


#for i in ls:
ts_test[f'target_profitable_{i}m'] = (grouped_predict['price'].shift(-i) > 1.02 * df_predict['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_test['zipcode'] = grouped_predict['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = df_predict.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)
        '''

"ts_test = pd.DataFrame()\n\n\n#for i in ls:\nts_test[f'target_profitable_{i}m'] = (grouped_predict['price'].shift(-i) > 1.02 * df_predict['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!\nts_test['zipcode'] = grouped_predict['zipcode'].shift(0)# .astype('object') # but remains still an integer...\nnumeric_column_names = df_predict.select_dtypes(include='number').columns # drop the lags of zipcode\n\nfor feature in numeric_column_names:\n    for lag in range(1, 4):  # 1 to 3 months\n        ts_test[f'{feature}_{lag}m'] = grouped_predict[f'{feature}'].shift(lag)\n        "